In [2]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [4]:
# Path to the template TSV file containing the headers
template_file_path = "node_template/submission_follow_up_template.tsv"

# Read the template TSV file to extract the headers
df_template = pd.read_csv(template_file_path, sep="\t", nrows=0)  # Read only the header
headers = df_template.columns.tolist()  # Extract the headers as a list

In [3]:
# Observational Patients
# File paths
case_path_obs = "/Users/jinn/Documents/IU/ARDaC/case_obs_DCC_data_release_v2-0-0.tsv"

# Read the files using pandas
df_obs_case = pd.read_csv(case_path_obs, sep="\t", dtype=str)

# Extract "*submitter_id" from df_obs_case and create case_table
case_table = pd.DataFrame()
case_table["*submitter_id"] = df_obs_case["*submitter_id"]
case_table["usubjid"] = case_table["*submitter_id"].apply(lambda x: x.split("_")[0])  # Extract the number before "_"

# Initialize the output DataFrame with the headers
df_output = pd.DataFrame(columns=headers)

In [4]:
# Define the extensions for mapping
extensions = {"Week 0": "_0", "Week 4": "_28", "Week 12": "_84", "Week 24": "_168"}

for _, row in tqdm(case_table.iterrows(), total=len(case_table), desc="Generating OBS Follow-up Rows"):
    submitter_id = row["*submitter_id"]

    for week, ext in extensions.items():
        new_row = {
            "*type": "follow_up",
            "project_id": "ARDaC-AlcHepNet",
            "*submitter_id": f"{submitter_id}{ext}",
            "cases.submitter_id": submitter_id,
            "demographics.submitter_id": f"{submitter_id}_demographic",
            "*days_to_follow_up": ext.lstrip("_"),  # Remove "_" from the extension
            "visit_day": ext.lstrip("_")
        }
        df_output = pd.concat([df_output, pd.DataFrame([new_row])], ignore_index=True)


enerating Follow-up Rows: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1134/1134 [04:49<00:00,  3.92it/s]

In [5]:
# OBS_LIVERSCORES
# Path to the liver scores file
liver_scores_path = "/Users/jinn/Documents/IU/ARDaC/DCC_data_release_v2.0.0/raw_data/Data for Nanxin/OBS Final Datasets/OBS_LIVERSCORES.csv"
df_liver_scores = pd.read_csv(liver_scores_path, sep=",", dtype=str)

# Process the liver scores and map values to follow-up rows
for _, row in tqdm(df_liver_scores.iterrows(), total=len(df_liver_scores), desc="Processing Liver Scores"):
    usubjid = row["usubjid"]
    redcap_event_name = row["redcap_event_name"]

    # Only process rows with valid weeks
    if redcap_event_name not in extensions:
        continue

    # Convert redcap_event_name to the extension
    extension = extensions[redcap_event_name]
    submitter_id = f"{usubjid}_obs{extension}"

    # Find the matching row in df_output
    output_row_index = df_output[df_output["*submitter_id"] == submitter_id].index
    if not output_row_index.empty:
        # Perform the mapping
        output_row_index = output_row_index[0]  # Get the first matching index
        df_output.loc[output_row_index, "meld_score"] = row.get("meld", None)
        df_output.loc[output_row_index, "child_pugh_score"] = row.get("cps", None)
        df_output.loc[output_row_index, "tlfb_drinking_days"] = row.get("tlfbnumdd", None)
        df_output.loc[output_row_index, "tlfb_number_drinks"] = row.get("tlfbnumd", None)
        df_output.loc[output_row_index, "liver_score_date"] = row.get("liverdat", None)


rocessing Liver Scores: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 2642/2642 [00:01<00:00, 1687.37it/s]

In [6]:
# OBS_MEDINFO
# Path to the liver scores file
med_info_path = "/Users/jinn/Documents/IU/ARDaC/DCC_data_release_v2.0.0/raw_data/Data for Nanxin/OBS Final Datasets/OBS_MEDINFO.csv"
df_med_info = pd.read_csv(med_info_path, sep=",", dtype=str)

# Process the medinfo and map values to follow-up rows
for _, row in tqdm(df_med_info.iterrows(), total=len(df_med_info), desc="Processing MedInfo"):
    usubjid = row["usubjid"]
    redcap_event_name = row["redcap_event_name"]

    # Only process rows with valid weeks
    if redcap_event_name not in extensions:
        continue

    # Convert redcap_event_name to the extension
    extension = extensions[redcap_event_name]
    submitter_id = f"{usubjid}_obs{extension}"

    # Find the matching row in df_output
    output_row_index = df_output[df_output["*submitter_id"] == submitter_id].index
    if not output_row_index.empty:
        # Perform the mapping
        output_row_index = output_row_index[0]  # Get the first matching index
        df_output.loc[output_row_index, "ascites_culture"] = row.get("ascyn", None)
        df_output.loc[output_row_index, "hep_enceph"] = row.get("hepenyn", None)
        df_output.loc[output_row_index, "varices"] = row.get("varyn", None)
        df_output.loc[output_row_index, "hep_carcinoma"] = row.get("hepcaryn", None)
        df_output.loc[output_row_index, "liver_transplant"] = row.get("livtnsplyn", None)
        df_output.loc[output_row_index, "ascites_date"] = row.get("ascdat", None)
        df_output.loc[output_row_index, "hep_enceph_diagnosis_date"] = row.get("hependat", None)
        df_output.loc[output_row_index, "varices_diagnosis_date"] = row.get("vardat", None)
        df_output.loc[output_row_index, "hepcar_diagnosis_date"] = row.get("hepcardat", None)
        df_output.loc[output_row_index, "liver_transplant_date"] = row.get("livtnspldat", None)


rocessing MedInfo: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1878/1878 [00:02<00:00, 892.74it/s]

In [7]:
# OBS_VITALS

# Path to the vitals file
vitals_path = "/Users/jinn/Documents/IU/ARDaC/DCC_data_release_v2.0.0/raw_data/Data for Nanxin/OBS Final Datasets/OBS_VITALS.csv"
df_vitals = pd.read_csv(vitals_path, sep=",", dtype=str)

# Process the medinfo and map values to follow-up rows
for _, row in tqdm(df_vitals.iterrows(), total=len(df_vitals), desc="Processing Vitals"):
    usubjid = row["usubjid"]
    redcap_event_name = row["redcap_event_name"]

    # Only process rows with valid weeks
    if redcap_event_name not in extensions:
        continue

    # Convert redcap_event_name to the extension
    extension = extensions[redcap_event_name]
    submitter_id = f"{usubjid}_obs{extension}"

    # Find the matching row in df_output
    output_row_index = df_output[df_output["*submitter_id"] == submitter_id].index
    if not output_row_index.empty:
        # Perform the mapping
        output_row_index = output_row_index[0]  # Get the first matching index
        df_output.loc[output_row_index, "weight"] = row.get("weight", None)
        df_output.loc[output_row_index, "bmi"] = row.get("bmi", None)


rocessing Vitals: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1829/1829 [00:00<00:00, 2507.56it/s]

In [8]:
# OBS_SOC

# Path to the SOC file
soc_path = "/Users/jinn/Documents/IU/ARDaC/DCC_data_release_v2.0.0/raw_data/Data for Nanxin/OBS Final Datasets/OBS_SOC.csv"
df_soc = pd.read_csv(soc_path, sep=",", dtype=str)

# Process the medinfo and map values to follow-up rows
for _, row in tqdm(df_soc.iterrows(), total=len(df_soc), desc="Processing SOC"):
    usubjid = row["usubjid"]
    redcap_event_name = row["redcap_event_name"]

    # Only process rows with valid weeks
    if redcap_event_name not in extensions:
        continue

    # Convert redcap_event_name to the extension
    extension = extensions[redcap_event_name]
    submitter_id = f"{usubjid}_obs{extension}"

    # Find the matching row in df_output
    output_row_index = df_output[df_output["*submitter_id"] == submitter_id].index
    if not output_row_index.empty:
        # Perform the mapping
        output_row_index = output_row_index[0]  # Get the first matching index
        df_output.loc[output_row_index, "infection_screen_done"] = row.get("infscreennd", None)
        df_output.loc[output_row_index, "infection_screen_date"] = row.get("infscreen_date", None)
        df_output.loc[output_row_index, "blood_culture"] = row.get("socisbcnd___999", None)
        df_output.loc[output_row_index, "blood_culture_result"] = row.get("socisbc", None)
        df_output.loc[output_row_index, "blood_organism"] = row.get("socisbc_pos", None)
        df_output.loc[output_row_index, "blood_culture_date"] = row.get("socisbcdat", None)
        df_output.loc[output_row_index, "urine_culture"] = row.get("socisucnd___999", None)
        df_output.loc[output_row_index, "urine_culture_result"] = row.get("socisuc", None)
        df_output.loc[output_row_index, "urine_culture_organism"] = row.get("socisuc_pos", None)
        df_output.loc[output_row_index, "urine_culture_date"] = row.get("socisucdat", None)
        df_output.loc[output_row_index, "urine_culture_fungal_result"] = row.get("soicuc_fung", None)
        df_output.loc[output_row_index, "ascites_culture"] = row.get("socisacnd___999", None)
        df_output.loc[output_row_index, "ascites_culture_result"] = row.get("socisac", None)
        df_output.loc[output_row_index, "ascites_organism"] = row.get("socisac_pos", None)
        df_output.loc[output_row_index, "ascites_date"] = row.get("socisacdat", None)
        df_output.loc[output_row_index, "endoscopy"] = row.get("endond", None)
        df_output.loc[output_row_index, "endoscopy_date"] = row.get("endodat", None)
        df_output.loc[output_row_index, "esophageal_varices_size"] = row.get("endovarsiz_esoph", None)
        df_output.loc[output_row_index, "esophageal_varices_bleed"] = row.get("endobled_esoph", None)
        df_output.loc[output_row_index, "gastric_varices_size"] = row.get("endovarsiz_gast", None)
        df_output.loc[output_row_index, "gastric_varices_bleed"] = row.get("endobled_gast", None)
        df_output.loc[output_row_index, "portal_hypertensive_gastropathy"] = row.get("porthypsev", None)
        df_output.loc[output_row_index, "esophageal_ulcer_size"] = row.get("endoulcsiz_esoph", None)
        df_output.loc[output_row_index, "esophageal_ulcer_bleed"] = row.get("endoulcbled_esoph", None)
        df_output.loc[output_row_index, "gastric_ulcer_size"] = row.get("endoulcsiz_gast", None)
        df_output.loc[output_row_index, "gastric_ulcer_bleed"] = row.get("endoulcbled_gast", None)
        df_output.loc[output_row_index, "duodenum_ulcer_size"] = row.get("endoulcsiz_duod", None)
        df_output.loc[output_row_index, "duodenum_ulcer_bleed"] = row.get("endoulcbled_duod", None)



rocessing SOC: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1382/1382 [00:03<00:00, 368.38it/s]

In [9]:
# Final check: Remove empty rows and log them in a QC file
qc_records = []  # To store QC information for deleted rows

# Columns that define a row as "non-empty" (exclude the fixed columns)
non_empty_columns = list(set(df_output.columns) - {
    "*type", "project_id", "*submitter_id", "cases.submitter_id",
    "demographics.submitter_id", "*days_to_follow_up", "visit_day"
})

# Iterate over df_output rows to identify empty rows
for index, row in df_output.iterrows():
    # Check if all non-fixed columns are empty
    if row[non_empty_columns].isnull().all():
        # Record the QC information
        qc_records.append({
            "usubjid": row["cases.submitter_id"].split("_")[0],  # Extract usubjid
            "*submitter_id": row["*submitter_id"],
            "empty_follow-up": "Y"
        })
        # Drop the empty row
        df_output.drop(index, inplace=True)

# Save the QC records to a TSV file
qc_output_path = "follow-up_qc_obs_DCC_data_release_v2-0-0.tsv"
df_qc = pd.DataFrame(qc_records)
df_qc.to_csv(qc_output_path, sep="\t", index=False, header=True)

# Save the cleaned df_output to a TSV file
output_path = "follow-up_obs_DCC_data_release_v2-0-0.tsv"
df_output.to_csv(output_path, sep="\t", index=False, header=True)

print(f"Cleaned follow-up table saved as: {output_path}")
print(f"QC table saved as: {qc_output_path}")

Cleaned follow-up table saved as: follow-up_obs_DCC_data_release_v2-0-0.tsv
QC table saved as: follow-up_qc_obs_DCC_data_release_v2-0-0.tsv


In [33]:
# Clinical Patients
# File paths
case_path_rct = "/Users/jinn/Documents/IU/ARDaC/case_rct_DCC_data_release_v2-0-0.tsv"

# Read the files using pandas
df_rct_case = pd.read_csv(case_path_rct, sep="\t", dtype=str)

# Extract "*submitter_id" from df_obs_case and create case_table
case_table_rct = pd.DataFrame()
case_table_rct["*submitter_id"] = df_rct_case["*submitter_id"]
case_table_rct["usubjid"] = case_table_rct["*submitter_id"].apply(lambda x: x.split("_")[0])  # Extract the number before "_"

# Initialize the output DataFrame with the headers
df_output_rct = pd.DataFrame(columns=headers)

In [34]:
# Define the extensions for mapping
extensions_rct = {"Day 0": "_0", "Day 3": "_3", "Day 7": "_7", "Day 14": "_14", "Day 28": "_28","Day 60": "_60","Day 90": "_90", "Day 180": "_180"}

for _, row in tqdm(case_table_rct.iterrows(), total=len(case_table_rct), desc="Generating RCT Follow-up Rows"):
    submitter_id = row["*submitter_id"]

    for day, ext in extensions_rct.items():
        new_row = {
            "*type": "follow_up",
            "project_id": "ARDaC-AlcHepNet",
            "*submitter_id": f"{submitter_id}{ext}",
            "cases.submitter_id": submitter_id,
            "demographics.submitter_id": f"{submitter_id}_demographic",
            "*days_to_follow_up": ext.lstrip("_"),  # Remove "_" from the extension
            "visit_day": ext.lstrip("_")
        }
        df_output_rct = pd.concat([df_output_rct, pd.DataFrame([new_row])], ignore_index=True)


enerating RCT Follow-up Rows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 147/147 [00:19<00:00,  7.43it/s]

In [39]:
# RCT_LIVERSCORES
# Path to the liver scores file
liver_scores_path_rct = "/Users/jinn/Documents/IU/ARDaC/DCC_data_release_v2.0.0/raw_data/Data for Nanxin/RCT Final Datasets/RCT_LIVERSCORES.csv"
df_liver_scores_rct = pd.read_csv(liver_scores_path_rct, sep=",", dtype=str)

# Process the liver scores and map values to follow-up rows
for _, row in tqdm(df_liver_scores_rct.iterrows(), total=len(df_liver_scores_rct), desc="Processing RCT Liver Scores"):
    usubjid = row["usubjid"]
    redcap_event_name = row["redcap_event_name"]

    # Only process rows with valid Days
    if redcap_event_name not in extensions_rct:
        continue

    # Convert redcap_event_name to the extension
    extension = extensions_rct[redcap_event_name]
    submitter_id = f"{usubjid}_clinical{extension}"

    # Find the matching row in df_output
    output_row_index = df_output_rct[df_output_rct["*submitter_id"] == submitter_id].index
    if not output_row_index.empty:
        # Perform the mapping
        output_row_index = output_row_index[0]  # Get the first matching index
        df_output_rct.loc[output_row_index, "meld_score"] = row.get("meld", None)
        df_output_rct.loc[output_row_index, "child_pugh_score"] = row.get("cps", None)
        df_output_rct.loc[output_row_index, "tlfb_drinking_days"] = row.get("tlfbnumdd", None)
        df_output_rct.loc[output_row_index, "tlfb_number_drinks"] = row.get("tlfbnumd", None)
        df_output_rct.loc[output_row_index, "liver_score_date"] = row.get("liverdat", None)


rocessing RCT Liver Scores: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 870/870 [00:00<00:00, 2120.55it/s]

In [41]:
# # RCT_MEDINFO
# # Path to the liver scores file
# med_info_path_rct = "/Users/jinn/Documents/IU/ARDaC/DCC_data_release_v2.0.0/raw_data/Data for Nanxin/RCT Final Datasets/OBS_MEDINFO.csv"
# df_med_info = pd.read_csv(med_info_path, sep=",", dtype=str)

# # Process the medinfo and map values to follow-up rows
# for _, row in tqdm(df_med_info.iterrows(), total=len(df_med_info), desc="Processing MedInfo"):
#     usubjid = row["usubjid"]
#     redcap_event_name = row["redcap_event_name"]

#     # Only process rows with valid weeks
#     if redcap_event_name not in extensions_rct:
#         continue

#     # Convert redcap_event_name to the extension
#     extension = extensions_rct[redcap_event_name]
#     submitter_id = f"{usubjid}_clinical{extension}"

#     # Find the matching row in df_output
#     output_row_index = df_output[df_output["*submitter_id"] == submitter_id].index
#     if not output_row_index.empty:
#         # Perform the mapping
#         output_row_index = output_row_index[0]  # Get the first matching index
#         df_output.loc[output_row_index, "ascites_culture"] = row.get("ascyn", None)
#         df_output.loc[output_row_index, "hep_enceph"] = row.get("hepenyn", None)
#         df_output.loc[output_row_index, "varices"] = row.get("varyn", None)
#         df_output.loc[output_row_index, "hep_carcinoma"] = row.get("hepcaryn", None)
#         df_output.loc[output_row_index, "liver_transplant"] = row.get("livtnsplyn", None)
#         df_output.loc[output_row_index, "ascites_date"] = row.get("ascdat", None)
#         df_output.loc[output_row_index, "hep_enceph_diagnosis_date"] = row.get("hependat", None)
#         df_output.loc[output_row_index, "varices_diagnosis_date"] = row.get("vardat", None)
#         df_output.loc[output_row_index, "hepcar_diagnosis_date"] = row.get("hepcardat", None)
#         df_output.loc[output_row_index, "liver_transplant_date"] = row.get("livtnspldat", None)

In [42]:
# RCT_VITALS

# Path to the vitals file
vitals_path_rct = "/Users/jinn/Documents/IU/ARDaC/DCC_data_release_v2.0.0/raw_data/Data for Nanxin/RCT Final Datasets/RCT_VITALS.csv"
df_vitals_rct = pd.read_csv(vitals_path_rct, sep=",", dtype=str)

# Process the medinfo and map values to follow-up rows
for _, row in tqdm(df_vitals_rct.iterrows(), total=len(df_vitals_rct), desc="Processing RCT Vitals"):
    usubjid = row["usubjid"]
    redcap_event_name = row["redcap_event_name"]

    # Only process rows with valid weeks
    if redcap_event_name not in extensions_rct:
        continue

    # Convert redcap_event_name to the extension
    extension = extensions_rct[redcap_event_name]
    submitter_id = f"{usubjid}_clinical{extension}"

    # Find the matching row in df_output
    output_row_index = df_output_rct[df_output_rct["*submitter_id"] == submitter_id].index
    if not output_row_index.empty:
        # Perform the mapping
        output_row_index = output_row_index[0]  # Get the first matching index
        df_output_rct.loc[output_row_index, "weight"] = row.get("weight", None)
        df_output_rct.loc[output_row_index, "bmi"] = row.get("bmi", None)


rocessing RCT Vitals: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 817/817 [00:00<00:00, 3677.95it/s]

In [43]:
# RCT_SOC

# Path to the SOC file
soc_path_rct = "/Users/jinn/Documents/IU/ARDaC/DCC_data_release_v2.0.0/raw_data/Data for Nanxin/RCT Final Datasets/RCT_SOC.csv"
df_soc_rct = pd.read_csv(soc_path_rct, sep=",", dtype=str)

# Process the medinfo and map values to follow-up rows
for _, row in tqdm(df_soc_rct.iterrows(), total=len(df_soc_rct), desc="Processing RCT SOC"):
    usubjid = row["usubjid"]
    redcap_event_name = row["redcap_event_name"]

    # Only process rows with valid weeks
    if redcap_event_name not in extensions_rct:
        continue

    # Convert redcap_event_name to the extension
    extension = extensions_rct[redcap_event_name]
    submitter_id = f"{usubjid}_clinical{extension}"

    # Find the matching row in df_output
    output_row_index = df_output_rct[df_output_rct["*submitter_id"] == submitter_id].index
    if not output_row_index.empty:
        # Perform the mapping
        output_row_index = output_row_index[0]  # Get the first matching index
        df_output_rct.loc[output_row_index, "infection_screen_done"] = row.get("infscreennd", None)
        df_output_rct.loc[output_row_index, "infection_screen_date"] = row.get("infscreen_date", None)
        df_output_rct.loc[output_row_index, "blood_culture"] = row.get("socisbcnd___999", None)
        df_output_rct.loc[output_row_index, "blood_culture_result"] = row.get("socisbc", None)
        df_output_rct.loc[output_row_index, "blood_organism"] = row.get("socisbc_pos", None)
        df_output_rct.loc[output_row_index, "blood_culture_date"] = row.get("socisbcdat", None)
        df_output_rct.loc[output_row_index, "urine_culture"] = row.get("socisucnd___999", None)
        df_output_rct.loc[output_row_index, "urine_culture_result"] = row.get("socisuc", None)
        df_output_rct.loc[output_row_index, "urine_culture_organism"] = row.get("socisuc_pos", None)
        df_output_rct.loc[output_row_index, "urine_culture_date"] = row.get("socisucdat", None)
        df_output_rct.loc[output_row_index, "urine_culture_fungal_result"] = row.get("soicuc_fung", None)
        df_output_rct.loc[output_row_index, "ascites_culture"] = row.get("socisacnd___999", None)
        df_output_rct.loc[output_row_index, "ascites_culture_result"] = row.get("socisac", None)
        df_output_rct.loc[output_row_index, "ascites_organism"] = row.get("socisac_pos", None)
        df_output_rct.loc[output_row_index, "ascites_date"] = row.get("socisacdat", None)
        df_output_rct.loc[output_row_index, "endoscopy"] = row.get("endond", None)
        df_output_rct.loc[output_row_index, "endoscopy_date"] = row.get("endodat", None)
        df_output_rct.loc[output_row_index, "esophageal_varices_size"] = row.get("endovarsiz_esoph", None)
        df_output_rct.loc[output_row_index, "esophageal_varices_bleed"] = row.get("endobled_esoph", None)
        df_output_rct.loc[output_row_index, "gastric_varices_size"] = row.get("endovarsiz_gast", None)
        df_output_rct.loc[output_row_index, "gastric_varices_bleed"] = row.get("endobled_gast", None)
        df_output_rct.loc[output_row_index, "portal_hypertensive_gastropathy"] = row.get("porthypsev", None)
        df_output_rct.loc[output_row_index, "esophageal_ulcer_size"] = row.get("endoulcsiz_esoph", None)
        df_output_rct.loc[output_row_index, "esophageal_ulcer_bleed"] = row.get("endoulcbled_esoph", None)
        df_output_rct.loc[output_row_index, "gastric_ulcer_size"] = row.get("endoulcsiz_gast", None)
        df_output_rct.loc[output_row_index, "gastric_ulcer_bleed"] = row.get("endoulcbled_gast", None)
        df_output_rct.loc[output_row_index, "duodenum_ulcer_size"] = row.get("endoulcsiz_duod", None)
        df_output_rct.loc[output_row_index, "duodenum_ulcer_bleed"] = row.get("endoulcbled_duod", None)


rocessing RCT SOC: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 429/429 [00:00<00:00, 527.21it/s]

In [44]:
# Final check: Remove empty rows and log them in a QC file
qc_records_rct = []  # To store QC information for deleted rows

# Columns that define a row as "non-empty" (exclude the fixed columns)
non_empty_columns = list(set(df_output_rct.columns) - {
    "*type", "project_id", "*submitter_id", "cases.submitter_id",
    "demographics.submitter_id", "*days_to_follow_up", "visit_day"
})

# Iterate over df_output rows to identify empty rows
for index, row in df_output_rct.iterrows():
    # Check if all non-fixed columns are empty
    if row[non_empty_columns].isnull().all():
        # Record the QC information
        qc_records_rct.append({
            "usubjid": row["cases.submitter_id"].split("_")[0],  # Extract usubjid
            "*submitter_id": row["*submitter_id"],
            "empty_follow-up": "Y"
        })
        # Drop the empty row
        df_output_rct.drop(index, inplace=True)

# Save the QC records to a TSV file
qc_output_path_rct = "follow-up_qc_rct_DCC_data_release_v2-0-0.tsv"
df_qc_rct = pd.DataFrame(qc_records_rct)
df_qc_rct.to_csv(qc_output_path_rct, sep="\t", index=False, header=True)

# Save the cleaned df_output to a TSV file
output_path_rct = "follow-up_rct_DCC_data_release_v2-0-0.tsv"
df_output_rct.to_csv(output_path_rct, sep="\t", index=False, header=True)

print(f"Cleaned RCT follow-up table saved as: {output_path_rct}")
print(f"RCT QC table saved as: {qc_output_path_rct}")

Cleaned RCT follow-up table saved as: follow-up_rct_DCC_data_release_v2-0-0.tsv
RCT QC table saved as: follow-up_qc_rct_DCC_data_release_v2-0-0.tsv
